Stage 1: Data collection
- Create CV file where any stock entered data is automatically parsed and saved

In [4]:
import csv
import yfinance as yf

# Step 1: Create or locate the CSV file
csv_file = "StockData.csv"

try:
    # Check if the file already exists
    with open(csv_file, "r"):
        pass
except FileNotFoundError:
    # Create a new file with header if it doesn't exist
    with open(csv_file, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Ticker Symbol", "Average volume", "t-13 vol", "t-13 change", "t-12 vol", "t-12 change", "t-11 vol", "t-11 change", "t-10 vol", "t-10 change", "t-9 vol", "t-9 change", "t-8 vol", "t-8 change", "t-7 vol", "t-7 change","t-6 vol", "t-6 change","t-5 vol", "t-5 change","t-4 vol", "t-4 change", "t-3 vol",
                         "t-3 change", "t-2 volume",
                         "t-2 change", "t-1 volume",
                         "t-1 change", "Result change"])

# Step 2: Take input of ticker symbol from user
ticker = input("Enter the ticker symbol: ")

# Step 3: Use Yahoo Finance API to search for the ticker
try:
    stock = yf.Ticker(ticker)
    info = stock.history(period="14d")
    #print(info)

    # Step 4: Return error if ticker does not exist
    if info.empty:
        print("Ticker does not exist.")
    else:
        # Step 5: Create a new row in the CSV file
        with open(csv_file, "a", newline="") as file:
            writer = csv.writer(file)
            avgVolume = info["Volume"].mean()
            # Step 6: Add information to the row
            row = [
                ticker,
                info["Volume"].mean(),  # Average volume

                info["Volume"].iloc[0]/avgVolume,  # 13 days ago's volume
                (info["Close"].iloc[0] - info["Open"].iloc[0])/(info["Open"].iloc[0])*100,  # 13 days ago's daily % change

                info["Volume"].iloc[1]/avgVolume,  # 12 days ago's volume
                (info["Close"].iloc[1] - info["Open"].iloc[1])/(info["Open"].iloc[1])*100,  # 12 days ago's daily % change

                info["Volume"].iloc[2]/avgVolume,  # 11 day ago's volume
                (info["Close"].iloc[2] - info["Open"].iloc[2])/(info["Open"].iloc[2])*100,  # 11 day ago's daily % change

                info["Volume"].iloc[3]/avgVolume,  # 10 day ago's volume
                (info["Close"].iloc[3] - info["Open"].iloc[3])/(info["Open"].iloc[3])*100,  # 10 day ago's daily % change

                info["Volume"].iloc[4]/avgVolume,  
                (info["Close"].iloc[4] - info["Open"].iloc[4])/(info["Open"].iloc[4])*100, 

                info["Volume"].iloc[5]/avgVolume,  
                (info["Close"].iloc[5] - info["Open"].iloc[5])/(info["Open"].iloc[5])*100, 
                
                info["Volume"].iloc[6]/avgVolume,  
                (info["Close"].iloc[6] - info["Open"].iloc[6])/(info["Open"].iloc[6])*100, 

                info["Volume"].iloc[7]/avgVolume,  
                (info["Close"].iloc[7] - info["Open"].iloc[7])/(info["Open"].iloc[7])*100, 

                info["Volume"].iloc[8]/avgVolume,  
                (info["Close"].iloc[8] - info["Open"].iloc[8])/(info["Open"].iloc[8])*100, 

                info["Volume"].iloc[9]/avgVolume,  
                (info["Close"].iloc[9] - info["Open"].iloc[9])/(info["Open"].iloc[9])*100,

                info["Volume"].iloc[10]/avgVolume,  
                (info["Close"].iloc[10] - info["Open"].iloc[10])/(info["Open"].iloc[10])*100,

                info["Volume"].iloc[11]/avgVolume,  
                (info["Close"].iloc[11] - info["Open"].iloc[11])/(info["Open"].iloc[11])*100,

                info["Volume"].iloc[12]/avgVolume,  
                (info["Close"].iloc[12] - info["Open"].iloc[12])/(info["Open"].iloc[4])*100,    

               (info["Close"].iloc[13] - info["Open"].iloc[13])/(info["Open"].iloc[13])*100,  # 0 days ago daily %
            ]
            writer.writerow(row)
            print("Data saved successfully.")

except Exception as e:
    print("An error occurred:", str(e))


                  Open        High         Low       Close     Volume  \
Date                                                                    
2023-06-05  426.695004  428.030042  424.792069  425.519379   65460200   
2023-06-06  425.090971  426.993875  424.413464  426.445923   64022200   
2023-06-07  426.854394  428.030020  424.533000  424.971375   85373300   
2023-06-08  425.041149  428.010132  424.244122  427.541870   61952800   
2023-06-09  428.368766  430.391252  427.282804  428.308990   85742800   
2023-06-12  429.325234  432.274271  428.578009  432.194550   76104300   
2023-06-13  433.708939  435.711479  433.021490  435.043976   95899700   
2023-06-14  435.392688  437.435089  431.985332  435.562042  100612100   
2023-06-15  434.715191  442.257182  434.615585  440.962006  110303100   
2023-06-16  443.019989  443.609985  438.970001  439.459991  114121300   
2023-06-20  437.450012  438.369995  435.029999  437.179993   76160400   
2023-06-21  436.160004  436.989990  434.329987  434